<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Before-Running-the-Code" data-toc-modified-id="Before-Running-the-Code-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Before Running the Code</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Please-do-the-the-following:" data-toc-modified-id="Please-do-the-the-following:-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span><b>Please do the the following:</b></a></span></li></ul></li></ul></li><li><span><a href="#HMM-Analysis-for-Power-Sweep-Data" data-toc-modified-id="HMM-Analysis-for-Power-Sweep-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>HMM Analysis for Power Sweep Data</a></span></li></ul></div>

# Before Running the Code

<h3 style="color:red"><b>Please do the the following:</b></h3>

0. Verify and update all the info in `attenuation.json`
1. **ONLY** Update the following in `metainfo.json` 
```json
    "dateOfMeasurement" : 20221005,
    "eccosorb": "True",
    "KandL": "True",
    "rigidCables": "True",
    "JPA": "True",
    "TWPA": "True",
    "ClearingCoupler": "False",
    "Device_id": "NBR07",
    "f0": 4.2727,
```

---

# HMM Analysis for Power Sweep Data

`project_path` should be the root folder of where all the Alazar data is stored.

`interactive` is a `Boolean` parameter. Set to `True` to enable manual safety checks and `False` for more automated performance.


In [1]:
from AlazarPowerSweepData import *

%matplotlib qt

project_path = r"G:\Shared drives\Quasiparticles_2\NBR07_Oct2022\test_power_sweep" 
interactive = False

power_sweep_obj = AlazarPowerSweepData(project_path,interactive)

`createPdf` takes value `True` or `False`. Set to `True` if you want to visually see the downsampled IQ plots (determined by `avgTime`) to determine the **attenuation** below which the system goes non-linear. Set to `False` if the attenuation is known.

In [2]:
createPdf = False

power_sweep_obj.process_Alazar_Data(avgTime=2,plots=createPdf)

Creating figure paths.....
Reading and sorting data files.....
Reading and updating the metadata.....
Data loaded without creating IQ downsampled plots....


`intTime` is the starting integration time provied to the HMM fits.

`SNRmin` is the minimum SNR cut off constraint to the HMM program.

In [3]:
intTime=1
SNRmin=3

power_sweep_obj.start_HMM_fit(intTime=intTime, SNRmin= SNRmin)



==========	HMM ANALYSIS STARTED	==========

Attenuation below which the system goes non-linear: 2
Attenuation Configuration:


LO_power = 20
Splitter = -3
Fixed_attenuator_1 = -20
Fixed_attenuator_2 = -30
DA_fixed = -10
Fridge_attenuation = -75
Ecosorb = -3
MetaData: 


dateOfMeasurement = 20221005
eccosorb = True
KandL = True
rigidCables = True
JPA = True
TWPA = False
ClearingCoupler = False
Device_id = NBR07
f0 = 4.2727
Temp = 30.0
sampleRateMHz = 10
durationSeconds = 5
LOf = 4.27238
phi = 0.47

The power to the device is -123 dBM at the chosen attenuation 2

Number of Modes you want to fit: 3
Extracted Means:
[(-40.464819239544454, -24.489742972349006), (31.906778799552967, -53.689563439316714), (-7.81125656680635, -55.416434542201905)]

Gussed Covariance:
[[[5 1]
  [1 5]]

 [[5 1]
  [1 5]]

 [[5 1]
  [1 5]]]


Starting HMM Analysis.....


HMM fitting concluded on Alazar Data at attenuation: 2 ...
HMM fitting concluded on Alazar Data at attenuation: 4 ...
HMM fitting concluded on 

NotImplementedError: 